In [1]:
import os
import torch

from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torch import nn
from torch.nn import functional as F

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

# PyTorch模型创建

<img style="float: center;" src="images/26.png" width="70%">

## 模型创建步骤

LeNet为例：

<img style="float: center;" src="images/27.png" width="70%">

LeNet模型计算图，由边和节点组成，节点表示每个数据，边表示数据之间的运算。

LeNet是一个很大的网络，接收输入，经过运算得到输出，在其内部，又分为多个子网络层进行拼接组成，这些子网络层之间连接配合，最终完成运算。

构建模型两大要素：
1. 构建子模块（LeNet里的卷积层、池化层、全连接层）
2. 拼接子模块（将子模块按照一定的顺序、逻辑拼接起来得到最终模型）

LeNet模型源码：
<img style="float: center;" src="images/28.png" width="70%">

LeNet.py文件，里面有一个LeNet类，继承了**nn.Module**：
- \_\_init\_\_方法：实现各个子模块的构建
- forward方法：实现各个子模块的拼接

进入outputs=net(inputs)，这里是模型前向传播的部分：

<img style="float: center;" src="images/29.png" width="70%">

进入module.py里的一个\_\_call\_\_函数（LeNet继承Module），有一行调用LeNet的forward方法。

<img style="float: center;" src="images/30.png" width="70%">
<img style="float: center;" src="images/31.png" width="70%">

## nn.Module类

所有的模型，所有的网络层都继承这个类。

<img style="float: center;" src="images/32.png" width="70%">

troch.nn：PyTorch神经网络模块，包含各个建模子模块，协同工作

### nn.Parameter

PyTorch模型的参数需要被优化器训练，因此通常要设置参数为requires_grad=True的张量。

一个模型中，往往有很多参数，很难直接手动管理这些参数。

PyTorch将参数用nn.Parameter来表示，并且用nn.Module来管理其结构下的所有参数。

In [2]:
## nn.Parameter 具有 requires_grad = True 属性
w = nn.Parameter(torch.randn(2,2))
print(w)
print(w.requires_grad)   # True


## nn.ParameterList 可以将多个nn.Parameter组成一个列表
params_list = nn.ParameterList([nn.Parameter(torch.rand(8,i)) for i in range(1,3)])
print(params_list)
print(params_list[0].requires_grad)


## nn.ParameterDict 可以将多个nn.Parameter组成一个字典
params_dict = nn.ParameterDict({"a":nn.Parameter(torch.rand(2,2)),
                                "b":nn.Parameter(torch.zeros(2))})
print(params_dict)
print(params_dict["a"].requires_grad)

Parameter containing:
tensor([[-0.1303,  0.2809],
        [ 0.6670,  0.0332]], requires_grad=True)
True
ParameterList(
    (0): Parameter containing: [torch.FloatTensor of size 8x1]
    (1): Parameter containing: [torch.FloatTensor of size 8x2]
)
True
ParameterDict(
    (a): Parameter containing: [torch.FloatTensor of size 2x2]
    (b): Parameter containing: [torch.FloatTensor of size 2]
)
True


用Module把这些参数管理起来

In [3]:
# module.parameters()返回一个生成器，包括其结构下的所有parameters

module = nn.Module()
module.w = w
module.params_list = params_list
module.params_dict = params_dict

num_param = 0
for param in module.parameters():
    print(param,"\n")
    num_param = num_param + 1

print("number of Parameters =",num_param)

Parameter containing:
tensor([[-0.1303,  0.2809],
        [ 0.6670,  0.0332]], requires_grad=True) 

Parameter containing:
tensor([[0.3539],
        [0.0656],
        [0.0592],
        [0.5325],
        [0.2082],
        [0.7110],
        [0.8437],
        [0.0257]], requires_grad=True) 

Parameter containing:
tensor([[0.0170, 0.9528],
        [0.1430, 0.9269],
        [0.8691, 0.8577],
        [0.4014, 0.0664],
        [0.9361, 0.1149],
        [0.2581, 0.9138],
        [0.9068, 0.4615],
        [0.2562, 0.8905]], requires_grad=True) 

Parameter containing:
tensor([[0.2722, 0.5718],
        [0.6498, 0.1995]], requires_grad=True) 

Parameter containing:
tensor([0., 0.], requires_grad=True) 

number of Parameters = 5


一般建模过程中，通过继承nn.Module来构建块类，将所有需要学习的参数部分放在构造函数中

In [4]:
# 以下范例为Pytorch中nn.Linear的源码的简化版本
# 可以看到它将需要学习的参数放在了__init__构造函数中
# 并在forward中调用F.linear函数来实现计算逻辑
class Linear(nn.Module):
    __constants__ = ['in_features', 'out_features']

    def __init__(self, in_features, out_features, bias=True):
        super(Linear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.Tensor(out_features, in_features))
        if bias:
            self.bias = nn.Parameter(torch.Tensor(out_features))
        else:
            self.register_parameter('bias', None)

    def forward(self, input):
        return F.linear(input, self.weight, self.bias)

### nn.functional

nn.functional（引入：from torch.nn import functional as F）

包含各种功能组件的函数实现：
- 激活函数：F.relu, F.sigmoid, F.tanh, F.softmax
- 模型层：F.linear, F.conv2d, F.max_pool2d, F.dropout2d, F.embedding
- 损失函数：F.binary_cross_entropy, F.mse_loss, F.cross_entropy

为了便于参数管理，通过继承nn.Module转换为类的实现形式，并直接封装在nn模块下：
- 激活函数：nn.Relu, nn.Sigmoid, nn.Tanh, nn.Softmax
- 模型层：nn.Linear, nn.Conv2d, nn.Max_pool2d, nn.Embedding
- 损失函数：nn.BCELoss, nn.MSELoss, nn.CrossEntorpyLoss

表面上用nn建立的这些激活函数，层，损失函数，背后都在functional里具体实现。

### nn.Module

nn.Module模块包含所有网络层的基类，管理有关网络的属性。

nn.Module里有8个重要的属性，用于管理整个模型，以有序字典的形式存在：
- self.\_parameters = OrderedDict()：存储管理属于nn.Parameter类的属性（权值、偏置）
- self.\_modules = OrderedDict()：存储管理nn.Module类，比如LeNet中，构建子模块，卷积层，池化层就存储在modules中
- self.\_buffers = OrderedDict()：存储管理缓冲属性（如BN层中的running_mean，std等）
- self.\_backward_hooks = OrderedDict()：存储管理钩子函数
- self.\_forward_hooks = OrderedDict()
- self.\_forward_pre_hooks = OrderedDict()
- self.\_state_dict_hooks = OrderedDict()
- self.\_load_state_dict_pre_hooks = OrderedDict()

进入pythonnet=LeNet(classes=2)，进入LeNet，可以看到LeNet继承nn.Module，\_\_init\_\_方法中第一行实现父类函数调用（调用nn.Module初始化函数）
<img style="float: center;" src="images/33.png" width="70%">

进入nn.Module的\_\_init\_\_初始化方法，调用\_construct方法实现8个有序字典的初始化操作，LeNet有8个有序字典属性，去存储后面的变量和参数（都是空的）
<img style="float: center;" src="images/34.png" width="70%">

之后LeNet建立第一个子模块Conv2d卷积层，进入nn.Conv2d卷积层：
<img style="float: center;" src="images/35.png" width="70%">

发现class Conv2d(\_ConvNd)类继承于\_ConvNd，在Conv2d的初始化方法中，依然是先调用父类的初始化方法，进入这个类，发现\_ConvNd继承Module类，初始化方法中用到super调用父类的初始化方法，说明**Conv2d子模块是一个Module，也具有8个有序字典的属性**：
<img style="float: center;" src="images/36.png" width="70%">

可以看到LeNet的\_Modules有序字典中，记录了上一步创建的子模块信息：
<img style="float: center;" src="images/37.png" width="70%">

由于这个Conv2d也是一个Module，因此它也有8个有序字典，但是它的\_modules是空的，说明当前该Conv2d模块没有子模块，但是它的\_parameters字典有参数（包含权重和偏置）
<img style="float: center;" src="images/38.png" width="70%">

构建第二个网络层观察LeNet如何将这个子模块Conv2d存储到\_modules字典里面，进入nn.Conv2d：
<img style="float: center;" src="images/39.png" width="70%">

直接跳出来，观察\_modules是否有变化，可以发现\_modules字典中依然只有conv1，没有出现conv2，因为目前只是通过初始化函数实现一个Conv2d的实例化，还没有赋值给conv2（一个子模块的**初始化**和**赋值**分为两步，第一步初始化之后，会被一个函数进行拦截，然后再进行第二步赋值）：
<img style="float: center;" src="images/40.png" width="70%">

进入拦截函数\_\_setattr\_\_（注意，首先是进入\_\_init\_\_里去初始化再进入拦截函数），该方法接收一个value，判断其类型，如果是参数，就保存到参数的有序字典中，如果是Module，就保存到模型的有序字典里。这里Conv2d是一个Module，所以会存入到\_modules字典中，name是conv2：
<img style="float: center;" src="images/41.png" width="70%">

再跳回来，就会发现\_modules字典中有conv2：
<img style="float: center;" src="images/42.png" width="70%">

然后依据上面，一步一步运行下去

**总结：**
- 先有一个大的Module（例如LeNet）继承nn.Module基类
- 这个大的Module里又有很多子模块，这些子模块同样也是继承于nn.Module
- 构建每个子模块：
  - 这些Module的\_\_init\_\_方法中，调用父类初始化进行8个属性初始化
  - 被\_\_setattr\_\_方法通过判断value类型将其保存到相应的属性字典里，再进行赋值
- 一个个构建子模块，最终把整个大的Module构建完毕

**注意：**
- 一个module可以包含多个子module（LeNet包含卷积层、池化层、全连接层）
- 一个module相当于一个运算，必须实现forward()函数（从计算图角度理解）
- 每个module都有8个字典管理它的属性（最常用\_parameters，\_modules）

通常很少直接用nn.Parameter定义参数构建模型，而是通过拼装一些常用的模型层来构建模型。

这些模型继承自nn.Module的对象，本身也包括参数，属于要定义模块的子模块。

nn.Module提供一些方法管理这些子模块：
- children()：返回生成器，包括模块下的所有子模块
- named_children()：返回生成器，包括模块下的所有子模块以及它们的名字
- modules()：返回生成器，包括模块下的所有各个层级的模块，包括模块本身
- named_modules()：返回生成器，包括模块下的所有各个层级的模块以及它们的名字，包括模块本身



举例：
下面Net类建立一个神经网络，包含两个子模块，分别用模型容器建立

In [5]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        
        self.embedding = nn.Embedding(num_embeddings = 10000, embedding_dim = 3, padding_idx = 1)
        
        self.conv = nn.Sequential()
        self.conv.add_module("conv_1", nn.Conv1d(in_channels = 3, out_channels = 16, kernel_size = 5))
        self.conv.add_module("pool_1", nn.MaxPool1d(kernel_size = 2))
        self.conv.add_module("relu_1", nn.ReLU())
        self.conv.add_module("conv_2", nn.Conv1d(in_channels = 16, out_channels = 128, kernel_size = 2))
        self.conv.add_module("pool_2", nn.MaxPool1d(kernel_size = 2))
        self.conv.add_module("relu_2", nn.ReLU())
        
        self.dense = nn.Sequential()
        self.dense.add_module("flatten", nn.Flatten())
        self.dense.add_module("linear", nn.Linear(6144, 1))
        self.dense.add_module("sigmoid", nn.Sigmoid())
        
    def forward(self,x):
        x = self.embedding(x).transpose(1, 2)
        x = self.conv(x)
        y = self.dense(x)
        return y


net = Net()

In [6]:
i = 0
for child in net.children():
    i += 1
    print(child,"\n")

print("child number",i)

Embedding(10000, 3, padding_idx=1) 

Sequential(
  (conv_1): Conv1d(3, 16, kernel_size=(5,), stride=(1,))
  (pool_1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu_1): ReLU()
  (conv_2): Conv1d(16, 128, kernel_size=(2,), stride=(1,))
  (pool_2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu_2): ReLU()
) 

Sequential(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear): Linear(in_features=6144, out_features=1, bias=True)
  (sigmoid): Sigmoid()
) 

child number 3


In [7]:
# 如果是net.named_children()，则会在该模块前加上conv:，dense:，表示变量的名字
i = 0
for child in net.named_children():
    i += 1
    print(child,"\n")

print("child number",i)

('embedding', Embedding(10000, 3, padding_idx=1)) 

('conv', Sequential(
  (conv_1): Conv1d(3, 16, kernel_size=(5,), stride=(1,))
  (pool_1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu_1): ReLU()
  (conv_2): Conv1d(16, 128, kernel_size=(2,), stride=(1,))
  (pool_2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu_2): ReLU()
)) 

('dense', Sequential(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear): Linear(in_features=6144, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)) 

child number 3


通过named_children方法找到embedding层，并将其参数设置为不可训练（相当于冻结embedding层，在迁移学习中经常用到）

In [8]:
children_dict = {name:module for name,module in net.named_children()}

print(children_dict)
embedding = children_dict["embedding"]
embedding.requires_grad_(False) #冻结其参数


#可以看到其第一层的参数已经不可以被训练了。
for param in embedding.parameters():
    print(param.requires_grad)    # False
    print(param.numel())   # 30000

{'embedding': Embedding(10000, 3, padding_idx=1), 'conv': Sequential(
  (conv_1): Conv1d(3, 16, kernel_size=(5,), stride=(1,))
  (pool_1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu_1): ReLU()
  (conv_2): Conv1d(16, 128, kernel_size=(2,), stride=(1,))
  (pool_2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu_2): ReLU()
), 'dense': Sequential(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear): Linear(in_features=6144, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)}
False
30000


# 提取层结构

给定一个模型，希望提取网络模型中的某一层或某几层

In [9]:
# 定义神经网络的结构
import torch
import torch.nn as nn


class SimpleCnn(nn.Module):
    def __init__(self):
        super(SimpleCnn, self).__init__()

        layer1 = nn.Sequential()
        layer1.add_module("conv1", nn.Conv2d(3, 32, 3, 1, padding=1))
        layer1.add_module("relu1", nn.ReLU(True))
        layer1.add_module("pool1", nn.MaxPool2d(2, 2))
        self.layer1 = layer1

        layer2 = nn.Sequential()
        layer2.add_module("conv2", nn.Conv2d(32, 64, 3, 1, padding=1))
        layer2.add_module("relu2", nn.ReLU(True))
        layer2.add_module("pool2", nn.MaxPool2d(2, 2))
        self.layer2 = layer2

        layer3 = nn.Sequential()
        layer3.add_module("conv3", nn.Conv2d(64, 128, 3, 1, padding=1))
        layer3.add_module("relu3", nn.ReLU(True))
        layer3.add_module("pool3", nn.MaxPool2d(2, 2))
        self.layer3 = layer3

        layer4 = nn.Sequential()
        layer4.add_module("fc1", nn.Linear(2048, 512))
        layer4.add_module("fc_relu1", nn.ReLU(True))
        layer4.add_module("fc2", nn.Linear(512, 64))
        layer4.add_module("fc_relu2", nn.ReLU(True))
        layer4.add_module("fc3", nn.Linear(64, 10))
        self.layer4 = layer4

    def forward(self, x):
        conv1 = self.layer1(x)
        conv2 = self.layer2(conv1)
        conv3 = self.layer3(conv2)
        fc_input = conv3.view(conv3.shape[0], -1)
        fc_out = self.layer4(fc_input)

        return fc_out

## children()

返回下一级模块的迭代器，例如在上述定义的神经网络模型中，只会返回：
- self.layer1
- self.layer2
- self.layer3
- self.layer4

不会返回它们内部的东西

In [10]:
# model.children()返回的是一个生成器
model = SimpleCnn()
print(model.children())

<generator object Module.children at 0x0000023A08007B30>


In [11]:
# 对model.children()进行迭代，提取神经网络下一级模块
model = SimpleCnn()
for layer in model.children():
    print(layer)

Sequential(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU(inplace=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)
Sequential(
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU(inplace=True)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)
Sequential(
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3): ReLU(inplace=True)
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)
Sequential(
  (fc1): Linear(in_features=2048, out_features=512, bias=True)
  (fc_relu1): ReLU(inplace=True)
  (fc2): Linear(in_features=512, out_features=64, bias=True)
  (fc_relu2): ReLU(inplace=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)


In [12]:
# 对下一级模块继续进行迭代，得到下下级模块
model = SimpleCnn()
for layer in model.children():
    for sublayer in layer.children():
        print(sublayer)

Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Linear(in_features=2048, out_features=512, bias=True)
ReLU(inplace=True)
Linear(in_features=512, out_features=64, bias=True)
ReLU(inplace=True)
Linear(in_features=64, out_features=10, bias=True)


In [13]:
# 如果想提取神经网络的前两层，实现如下
model = SimpleCnn()
new_model = nn.Sequential(*list(model.children())[0:2])
print(new_model)

Sequential(
  (0): Sequential(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu1): ReLU(inplace=True)
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu2): ReLU(inplace=True)
    (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
)


## modules()

返回模型中的所有模块迭代器（可以访问到最内层，比如self.layer1.conv1模块）

输出结果：返回神经网络的所有模块，包括各种不同级的模块。

In [14]:
model = SimpleCnn()
for layer in model.modules():
    print(layer)

SimpleCnn(
  (layer1): Sequential(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu1): ReLU(inplace=True)
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu2): ReLU(inplace=True)
    (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu3): ReLU(inplace=True)
    (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer4): Sequential(
    (fc1): Linear(in_features=2048, out_features=512, bias=True)
    (fc_relu1): ReLU(inplace=True)
    (fc2): Linear(in_features=512, out_features=64, bias=True)
    (fc_relu2): ReLU(inplace=True)
    (fc3): Linear(in_features=64, out_features=10, bias=True)
  )
)
Sequential(
  (con

## named_children()和named_modules()

named_children()对应children()

named_modules()对应modules()

不仅返回模块的迭代器，还会返回网络层的名字

In [15]:
model = SimpleCnn()
for layer in model.named_modules():
    print(layer)

('', SimpleCnn(
  (layer1): Sequential(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu1): ReLU(inplace=True)
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu2): ReLU(inplace=True)
    (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu3): ReLU(inplace=True)
    (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer4): Sequential(
    (fc1): Linear(in_features=2048, out_features=512, bias=True)
    (fc_relu1): ReLU(inplace=True)
    (fc2): Linear(in_features=512, out_features=64, bias=True)
    (fc_relu2): ReLU(inplace=True)
    (fc3): Linear(in_features=64, out_features=10, bias=True)
  )
))
('layer1', S

In [16]:
# 如果希望提取模型中所有的卷积层，实现如下
model = SimpleCnn()
new_model = nn.Sequential()
for layer in model.named_modules():
    if isinstance(layer[1], nn.Conv2d):
        conv_name = layer[0].replace('.', '_')
        new_model.add_module(conv_name, layer[1])

print(new_model)

Sequential(
  (layer1_conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (layer2_conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (layer3_conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)


# 提取参数及自定义初始化

## 提取参数

nn.Module里有两个关于参数的属性：
- parameters()：给出一个网络的全部参数迭代器
- named_parameters()：给出网络层的名字和参数迭代器

In [17]:
# 获取模型的参数，通过para.grad获取梯度
model = SimpleCnn()
for para in model.parameters():
    print(para)
    print(para.grad)

Parameter containing:
tensor([[[[-0.0134,  0.0954, -0.1791],
          [ 0.0790,  0.0568,  0.0007],
          [ 0.1107,  0.0336, -0.0625]],

         [[-0.1052, -0.0538,  0.0104],
          [-0.0820,  0.0838, -0.0520],
          [-0.0211, -0.1083,  0.0504]],

         [[ 0.0856,  0.0182, -0.1153],
          [ 0.0920, -0.0526, -0.1255],
          [-0.1390,  0.1031, -0.0354]]],


        [[[ 0.0998,  0.1194,  0.1106],
          [ 0.1052,  0.0715,  0.1137],
          [-0.0572, -0.1069,  0.1135]],

         [[ 0.0709, -0.1619, -0.0686],
          [ 0.0354,  0.1674,  0.1819],
          [ 0.1790,  0.0567,  0.1735]],

         [[-0.1844, -0.0339, -0.0174],
          [-0.0206, -0.0086, -0.0391],
          [ 0.0660, -0.0086,  0.1117]]],


        [[[ 0.0147,  0.1720,  0.1625],
          [ 0.0530, -0.1765, -0.1857],
          [ 0.0168,  0.0270,  0.1241]],

         [[ 0.1625, -0.0613,  0.1075],
          [-0.0460, -0.0357,  0.0806],
          [-0.0471, -0.0342,  0.0984]],

         [[ 0.0758, -0

          [-0.0599,  0.1640, -0.1639]]]], requires_grad=True)
None
Parameter containing:
tensor([-0.0418, -0.0331,  0.1548,  0.0706,  0.0074, -0.1830, -0.1541,  0.0411,
         0.0848,  0.0242, -0.1385,  0.0856, -0.1605,  0.0234, -0.0767,  0.1763,
        -0.1337, -0.0122, -0.0343,  0.0284,  0.0249,  0.1079,  0.0189, -0.0101,
        -0.1403, -0.0985, -0.0308, -0.1336, -0.1028,  0.1672, -0.1242, -0.0523],
       requires_grad=True)
None
Parameter containing:
tensor([[[[ 3.4295e-02,  1.5955e-02,  9.3117e-03],
          [-4.2725e-02,  5.2042e-02, -4.4845e-05],
          [-2.8939e-02, -4.8821e-02, -1.3016e-03]],

         [[ 1.0474e-02,  5.8891e-02,  4.9041e-02],
          [ 1.5308e-02, -2.8555e-02,  1.0906e-02],
          [ 3.1127e-02, -3.0017e-02, -4.7422e-02]],

         [[ 3.0458e-02,  4.6793e-03,  3.7602e-02],
          [ 1.9441e-02,  5.7104e-02,  4.5636e-02],
          [-2.5599e-02,  4.6267e-02, -9.7768e-03]],

         ...,

         [[-8.0346e-03, -3.6429e-02,  3.4747e-02],
     

In [18]:
# 获取模型网络层的名字和参数
# model.named_parameters()迭代出形式为（名字，参数值）的元组
model = SimpleCnn()
for name, para in model.named_parameters():
    print(name)

layer1.conv1.weight
layer1.conv1.bias
layer2.conv2.weight
layer2.conv2.bias
layer3.conv3.weight
layer3.conv3.bias
layer4.fc1.weight
layer4.fc1.bias
layer4.fc2.weight
layer4.fc2.bias
layer4.fc3.weight
layer4.fc3.bias


## 自定义初始化

In [19]:
model = SimpleCnn()
for m in model.modules():
    if isinstance(m, nn.Conv2d):
        # nn.init.normal_(m.weight.data)
        # nn.init.xavier_normal_(m.weight.data)
        nn.init.kaiming_normal_(m.weight.data)
        m.bias.data.fill_(0)
    elif isinstance(m, nn.Linear):
        m.weight.data.normal_()

# 模型容器Containers

<img style="float: center;" src="images/43.png" width="70%">

Containers容器里包含3个子模块：
- nn.Sequential
- nn.ModuleList
- nn.ModuleDict

<img style="float: center;" src="images/58.png" width="70%">

## nn.Sequential

按顺序包装一组网络层

深度学习中，以全连接层为界限，将网络模型划分为：
- 特征提取模块
- 分类模块

举例：LeNet模型，把前半部分划分为特征提取部分，后面的全连接层为分类模块
<img style="float: center;" src="images/44.png" width="70%">

In [20]:
# 利用sequential包装一个LeNet
class LeNetSequential(nn.Module):
    def __init__(self, classes):
        super(LeNetSequential, self).__init__()

        layer1 = nn.Sequential()
        layer1.add_module("conv1", nn.Conv2d(3, 6, 5))
        layer1.add_module("relu1", nn.ReLU())
        layer1.add_module("pool1", nn.MaxPool2d(kernel_size=2, stride=2))
        layer1.add_module("conv2", nn.Conv2d(6, 16, 5))
        layer1.add_module("relu2", nn.ReLU())
        layer1.add_module("pool2", nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer1 = layer1
#         # 或者
#         self.layer1 = nn.Sequential(
#             nn.Conv2d(3, 6, 5),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2),
            
#             nn.Conv2d(6, 16, 5),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2)
#         )

        layer2 = nn.Sequential()
        layer2.add_module("fc1", nn.Linear(16 * 5 * 5, 120))
        layer2.add_module("fc_relu1", nn.ReLU())
        layer2.add_module("fc2", nn.Linear(120, 84))
        layer2.add_module("fc_relu2", nn.ReLU())
        layer2.add_module("fc3", nn.Linear(84, classes))
        self.layer2 = layer2

    def forward(self, x):
        conv1 = self.layer1(x)
        fc_input = conv1.view(conv1.shape[0], -1)
        fc_out = self.layer2(fc_input)

        return fc_out

通过代码调试，观察Sequential搭建的LeNet属性，以及Sequential机制：
<img style="float: center;" src="images/45.png" width="70%">

通过super进行初始化（这个类继承nn.Module，因此包含8个属性字典），一直stepover直至到一个Sequential完成的位置停下来：
<img style="float: center;" src="images/46.png" width="70%">

stepinto->stepout->stepinto，进入container.py的Sequential类，发现：

class Sequential(Module)：说明Sequential也是继承Module类（也包含8个属性字典）
<img style="float: center;" src="images/47.png" width="70%">

一直stepover直到第5个子模块结束，这样一个Sequential就建完了，通过stepout回到主程序，然后往下执行，把第一个Sequential构建完毕：
<img style="float: center;" src="images/49.png" width="70%">

Sequential构建机制：
- 依然是继承Module类，也是在\_\_init\_\_方法中先调用父类去初始化8个属性字典
- 之后在\_\_init\_\_里完成各个子模块的参数存储
- 至此，子模块构建完成

拼接子模块，通过**前向传播函数**完成：
<img style="float: center;" src="images/50.png" width="70%">

进入net函数，看看前向传播实现过程，进入module.py的\_\_call\_\_函数，在这里调用前向传播：
<img style="float: center;" src="images/51.png" width="70%">

之后跳到LeNetSequential类的前向传播函数，完成第一个Sequential（features）的前向传播，只用一句代码：x=self.features(x)，这句代码实现6个子模块的前向传播
<img style="float: center;" src="images/52.png" width="70%">

由于self.features是一个Sequential（继承于Module），因此步入之后，依然会跳到module.py的\_\_call\_\_函数，stepout到前向传播的那一行，然后步入Sequential的前向传播：
<img style="float: center;" src="images/53.png" width="70%">

在Sequential的前向传播里，会根据之前定义的\_module有序属性字典遍历，得到每个子模块，进行处理，此处是串联机制（上一层的输出会是下一层的输入），因此要注意上下模型输入和输出的对应关系，数据格式，形状大小。
<img style="float: center;" src="images/54.png" width="70%">

模型拼接：
- 完全是Sequential实现，在Sequential定义的时候，把每一层的子模块信息存入到它的\_modules有序字典中
- 前向传播的时候，Sequential的forward函数就会遍历这个字典，把每一层拿出来处理数据
- 串行，上一层的输出正好是下一层的输入
- 通过这个迭代就可以完成前向传播过程

完成调试后，得到LeNetSequential最终的结构：
<img style="float: center;" src="images/55.png" width="70%">

网络层有名字（例如conv1, conv2），可以通过名字去索引网络层（如果没有命名则会变成序号）

如果网络层成千上百的话，很难通过序号去索引网络层，这时需要对网络层进行一个命名：

In [21]:
class LeNetSequentialOrderDict(nn.Module):
    def __init__(self, classes):
        super(LeNetSequentialOrderDict, self).__init__()

        self.features = nn.Sequential(OrderedDict({
            'conv1': nn.Conv2d(3, 6, 5),
            'relu1': nn.ReLU(inplace=True),
            'pool1': nn.MaxPool2d(kernel_size=2, stride=2),

            'conv2': nn.Conv2d(6, 16, 5),
            'relu2': nn.ReLU(inplace=True),
            'pool2': nn.MaxPool2d(kernel_size=2, stride=2),
        }))

        self.classifier = nn.Sequential(OrderedDict({
            'fc1': nn.Linear(16*5*5, 120),
            'fc1_relu': nn.ReLU(),

            'fc2': nn.Linear(120, 84),
            'fc2_relu': nn.ReLU(inplace=True),

            'fc3': nn.Linear(84, classes),
        }))

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size()[0], -1)
        x = self.classifier(x)
        return x

这里Sequential包装就是一个有序的字典（网络名：网络层）

通过这个就可以对每一层网络进行命名（在定义的时候命名）可以看到，Sequential初始化方法里有个判断，if后面判断是否传入有序字典，之前Sequential里是各个层（不满足if条件，直接跳到下面的else，self.add_module(str(idx), module)直接用数字索引命名后加入到\_module有序参数字典中），而当构建有序字典，就应该执行self.add_module(key, module)，把key(网络名):value(网络层)存入到\_module有序参数字典中
<img style="float: center;" src="images/56.png" width="70%">

Sequential总结：
- nn.Sequential是nn.module的容器，用于按顺序包装一组网络层
- 顺序性：各网络层之间严格按照顺序构建，注意前后层数据的关系
- 自带forward：通过for循环依次执行前向传播运算

## nn.ModuleList

nn.ModuleList是nn.module的容器，用于包装一组网络层，以**迭代**方式调用网络层：
- append()：在ModuleList后添加网络层
- extend()：拼接两个ModuleList
- insert()：指定在ModuleList中的位置插入网络层

该方法类似于列表List，使用ModuleList循环迭代实现一个20个全连接层的网络的构建：

In [22]:
class ModuleList(nn.Module):
    def __init__(self):
        super(ModuleList, self).__init__()
        self.linears = nn.ModuleList([nn.Linear(10, 10) for i in range(20)])

    def forward(self, x):
        for i, linear in enumerate(self.linears):
            x = linear(x)
        return x

ModuleList构建网络层使用列表生成式构建，前向传播的时候遍历每一层，进行计算。

ModuleList初始化，把20个全连接层连起来：
<img style="float: center;" src="images/57.png" width="70%">

可以看到这个modules是一个列表，里面为20个全连接层，前向传播用for循环获取每个网络层。

nn.ModuleList可以迭代模型，索引模型

## nn.ModuleDict

nn.ModuleDict是nn.module的容器，用于包装一组网络层，以**索引**方式调用网络层：
- clear()：清空ModuleDict
- items(): 返回可迭代的键值对(key-value pairs)
- keys(): 返回字典的键(key)
- values(): 返回字典的值(value)
- pop(): 返回一对键值对，并从字典中删除

In [23]:
# 可以通过ModuleDict实现网络层的选取
class ModuleDict(nn.Module):
    def __init__(self):
        super(ModuleDict, self).__init__()
        # 通过choices选择卷积或者池化
        self.choices = nn.ModuleDict({
            'conv': nn.Conv2d(10, 10, 3),
            'pool': nn.MaxPool2d(3)
        })

        # 通过activations选择激活函数
        self.activations = nn.ModuleDict({
            'relu': nn.ReLU(),
            'prelu': nn.PReLU()
        })

    def forward(self, x, choice, act):
        x = self.choices[choice](x)
        x = self.activations[act](x)
        return x
    
net = ModuleDict()
fake_img = torch.randn((4, 10, 32, 32))
output = net(fake_img, 'conv', 'relu')    # 在这里可以选择我们的层进行组合
print(output)

tensor([[[[7.2252e-01, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           3.6374e-01, 5.8302e-01],
          [9.7997e-01, 0.0000e+00, 3.9922e-01,  ..., 1.2673e+00,
           2.5317e-01, 3.5334e-01],
          [4.1634e-01, 2.0954e-01, 7.0186e-01,  ..., 2.5332e-01,
           1.3793e-01, 3.1918e-01],
          ...,
          [5.5952e-01, 1.3809e-01, 2.1021e-01,  ..., 7.5475e-01,
           1.8340e+00, 0.0000e+00],
          [0.0000e+00, 5.4428e-01, 0.0000e+00,  ..., 0.0000e+00,
           7.9935e-01, 1.4326e+00],
          [8.0358e-03, 0.0000e+00, 0.0000e+00,  ..., 3.2859e-01,
           0.0000e+00, 3.6908e-01]],

         [[3.5996e-01, 0.0000e+00, 1.1538e+00,  ..., 4.2405e-01,
           0.0000e+00, 4.0880e-01],
          [3.9575e-01, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 9.3711e-01],
          [1.0538e+00, 0.0000e+00, 0.0000e+00,  ..., 2.3686e-02,
           0.0000e+00, 9.8347e-01],
          ...,
          [4.1381e-01, 0.0000e+00, 1.0395e-01,  ..., 3.4284

ModuleDict在选择网络层的时候很实用，比如要做时间序列预测的时候，往往会用到GRU和LSTM，可以通过这种方式来对比哪种网络效果好。

# AlexNet构建

2012年ImageNet分类任务中获得冠军，开创卷积神经网络新时代

AlexNet特点：
- 采用ReLu：替换饱和激活函数，减轻梯度消失
- 采用LRN（Local Response Normalization）：对数据归一化，减轻梯度消失（之后被BN归一化取代）
- Dropout：提高全连接层鲁棒性，增加网络泛化能力
- Data Augmentation：TenCrop，色彩修改

AlexNet结构：
<img style="float: center;" src="images/59.png" width="70%">

In [24]:
class AlexNet(nn.Module):

    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        # 第一部分是一个Sequential，由一系列全集池化模块构成
        # 目的是提取图像的特征
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        
        # 全局池化层把特征进行整合
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        
        # Sequential是全连接层组成，用于模型分类
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# 思维导图

<img style="float: center;" src="images/60.png" width="70%">